In [7]:
import pandas as pd

def normalizeCSV(fileName):
    with open(fileName , "r") as file:
        inputFilelines = file.readlines()
        with open(fileName , "w") as file:
            for line in inputFilelines:
                if line != "\n" and line != "[Name]\n" and line != "WALL\n" and line != "\n" and line != "[Data]\n":
                    file.write(line)
    file.close()

for i in range(732):
    normalizeCSV(f"Wall/export_{i}.csv")

In [8]:
wss = {"Wall Shear" : [] , "Wall Shear x" : [] , "Wall Shear y" : [] , "Wall Shear z" : []}
for i in range(732):
    df = pd.read_csv(f"Wall/export_{i}.csv")
    wss["Wall Shear"].append(df[" Wall Shear [ kPa ]"][0:2888].values)
    wss["Wall Shear x"].append(df[" Wall Shear X [ kPa ]"][0:2888].values)
    wss["Wall Shear y"].append(df[" Wall Shear Y [ kPa ]"][0:2888].values)
    wss["Wall Shear z"].append(df[" Wall Shear Z [ kPa ]"][0:2888].values)
wss

{'Wall Shear': [array([0., 0., 0., ..., 0., 0., 0.]),
  array([0.00190247, 0.00243131, 0.00262095, ..., 0.00014239, 0.00014447,
         0.00014359]),
  array([0.00268859, 0.0034825 , 0.0037652 , ..., 0.00022601, 0.00022778,
         0.00022756]),
  array([0.00325921, 0.00434161, 0.00471621, ..., 0.00028548, 0.00028699,
         0.00028736]),
  array([0.0037748 , 0.00518851, 0.0056605 , ..., 0.00033424, 0.00033572,
         0.00033652]),
  array([0.00425341, 0.00602506, 0.00659622, ..., 0.00037684, 0.00037834,
         0.00037947]),
  array([0.00469633, 0.00685295, 0.00752378, ..., 0.00041541, 0.00041692,
         0.00041829]),
  array([0.00510928, 0.00768675, 0.00845784, ..., 0.00045116, 0.00045268,
         0.00045426]),
  array([0.00549961, 0.00854297, 0.00941557, ..., 0.00048489, 0.00048643,
         0.00048821]),
  array([0.00586859, 0.00942354, 0.01039625, ..., 0.00051688, 0.00051841,
         0.0005204 ]),
  array([0.00621984, 0.01033584, 0.01140921, ..., 0.00054748, 0.00054899,

In [9]:
def tawssCalculation(name , nodeNumber):
    result = 0
    for i in range(732):
        result += wss[name][i][nodeNumber]
    return result / 731


In [10]:
tawssValues = {}
osiValues = {}
holmesValues = {}
for nodeIndex in range(2888):
    tawssValues[f"Node {nodeIndex}"] = tawssCalculation("Wall Shear" , nodeIndex)
    
    wss_x = tawssCalculation("Wall Shear x" , nodeIndex)
    wss_y = tawssCalculation("Wall Shear y" , nodeIndex)
    wss_z = tawssCalculation("Wall Shear z" , nodeIndex)
    
    osiValues[f"Node {nodeIndex}"] = (1 - ((wss_x ** 2 + wss_y ** 2 + wss_z ** 2) ** 0.5) / tawssValues[f"Node {nodeIndex}"]) * 0.5
    
    holmesValues[f"Node {nodeIndex}"] = tawssValues[f"Node {nodeIndex}"] * (0.5 - osiValues[f"Node {nodeIndex}"])


In [24]:
output = pd.DataFrame({"TAWSS" : tawssValues.values() , "OSI" : osiValues.values() , "HOLMES" : holmesValues.values()})
index = []
for nodeIndex in range(len(wss["Wall Shear"][0])):
    index.append(f"Node {nodeIndex}")
output.index = index
output

,TAWSS,OSI,HOLMES
Node 0,0.005596,0.491104,0.000050
Node 1,0.011177,0.161577,0.003782
Node 2,0.011296,0.146239,0.003996
Node 3,0.005166,0.212339,0.001486
Node 4,0.006912,0.170655,0.002276
...,...,...,...
Node 2883,0.000674,0.000031,0.000337
Node 2884,0.000683,0.000005,0.000341
Node 2885,0.000689,0.000002,0.000344
Node 2886,0.000659,0.000006,0.000329


In [29]:
outputFile = open("output.csv" , "a")
output.to_csv(outputFile)
outputFile.close()